In [26]:
import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
import numpy as np

In [27]:
sentences = ["I ate dinner.", "We had a three-course meal.", "Brad came to dinner with us.", "He loves fish tacos.","In the end, we all felt like we ate too much.","We all agreed; it was a magnificent evening."]

# Tokenization of each document
# tokenized_sent = []
# for s in sentences:
#     tokenized_sent.append(word_tokenize(s.lower()))
# print(tokenized_sent)

In [28]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [29]:

# import gensim
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
# tagged_data

In [30]:
## Train doc2vec model
# model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)

# '''
# vector_size = Dimensionality of the feature vectors.
# window = The maximum distance between the current and predicted word within a sentence.
# min_count = Ignores all words with total frequency lower than this.
# alpha = The initial learning rate.
# '''

## Print model vocabulary
# model.wv.vocab

In [31]:
# test_doc = word_tokenize("I had pizza and pasta".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])

# '''
# positive = List of sentences that contribute positively.
# '''

## SentenceBERT

In [32]:
# !pip install sentence-transformers

In [33]:

# from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [34]:
# sentence_embeddings = sbert_model.encode(sentences)

# print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
# print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])
# sentence_embeddings.shape

In [35]:
# # keep the first 100 embeddings for each sentence in a separate list
# embeddings = [sentence_embeddings[i][:100] for i in range(len(sentence_embeddings))]
# embeddings[0]

In [36]:
import pandas as pd

In [37]:
# Read csv file train.csv and store it in a variable called train_data
train_data = pd.read_csv('../input/feedback-prize-effectiveness/train.csv')
train_data.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [38]:
# create a list of sentences from the train_data - train_data['discourse_text']
train_sentences = train_data['discourse_text'].tolist()

discourse_type = train_data['discourse_type'].tolist()
# discourse_type

In [39]:
import csv

def readCSV(filename):
    rows = []
    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        for row in csvreader:
            rows.append(row)
    return rows

In [40]:
discourses = []
dis_types = []
dis_effectiveness = []
rows = readCSV("../input/feedback-prize-effectiveness/train.csv")
for row in rows:
    if len(row[2])<=510:
        discourses.append(row[2])
        dis_types.append(row[3])
        dis_effectiveness.append(row[4])
    while len(row[2])>510:
        temp = row[2][:510]
        row[2] = row[2][510:]
        discourses.append(temp)
        dis_types.append(row[3])
        dis_effectiveness.append(row[4])
print(len(discourses),discourses[0])
print(len(dis_types))
print(len(dis_effectiveness))

37762 Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. 
37762
37762


In [41]:
typeDict = {'Lead':0,'Position':1,'Claim':2,'Counterclaim':3,'Rebuttal':4,'Evidence':5,'Concluding Statement':6}
effectDict = {'Ineffective':0,'Adequate':1,'Effective':2}

In [42]:
# dis_types

In [43]:
# %pip install sent2vec

In [44]:
import numpy as np
import os
import gensim
import torch
import transformers as ppb
# from sent2vec.splitter import *

import os
import re
import spacy

ROOT_DIR = os.getcwd() 
TEST_DIR = os.path.join(ROOT_DIR, 'test')
DATA_DIR = os.path.join(TEST_DIR, 'dataset')

ROOT_LOCAL = '/Users/pedramataee/'
ROOT_LINUX = '~'

WIKI_PATH = os.path.join(ROOT_LINUX, 'gensim-data/glove-wiki-gigaword-300') 
PRETRAINED_VECTORS_PATH_WIKI = os.path.join(WIKI_PATH, 'glove-wiki-gigaword-300.gz')

FASTTEXT_NEWS_PATH = os.path.join(ROOT_LINUX, 'gensim-data/fasttext-wiki-news-subwords-300') 
PRETRAINED_VECTORS_PATH_FASTTEXT = os.path.join(FASTTEXT_NEWS_PATH, 'fasttext-wiki-news-subwords-300.gz')

# Make sure to download "en_core_web_sm" package on your machine.
# In case, you can run: "python3 -m spacy download en_core_web_sm"
os.environ['LANGUAGE_MODEL_SPACY'] = "en_core_web_sm"


class Splitter:
    def __init__(self):
        self.words = []
        self.sentences = []
        try:
            self.nlp = spacy.load(os.environ['LANGUAGE_MODEL_SPACY'])
        except Exception as error:
            print(f'{error}\n\n Install "en_core_web_sm" in your environment. '
                  f'Try running: "python3 -m spacy download en_core_web_sm" \n '
                  f'or follow instrucions here: https://spacy.io/usage')

    def sent2words(self, sentences, **kwargs):
        add_stop_words = kwargs.get('add_stop_words', [])
        remove_stop_words = kwargs.get('remove_stop_words', [])

        for w in add_stop_words:
            self.nlp.vocab[w].is_stop = True
        for w in remove_stop_words:
            self.nlp.vocab[w].is_stop = False

        words = []
        for sentence in sentences:
            doc = self.nlp(sentence.lower())
            words.append([token.lemma_ for token in doc if not token.is_punct | token.is_space | token.is_stop])

        self.words = words

    def text2sents(self, texts):
        for text in texts:
            doc = self.nlp(text)
            span = doc[0:5]
            sents = list(doc.sents)
            self.sentences.extend([sent for sent in sents])

    def text2words(self, texts):
        doc = self.nlp(texts)
        tokenized_texts = []
        for w in doc:
            is_clean = w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num
            if is_clean:
                tokenized_texts.append(w.lemma_)

        self.words = tokenized_texts


def sentencizer_by_regex(texts):
    alphabets = "([A-Za-z])"
    prefixes = r"(Mr|St|Mrs|Ms|Dr)[.]"
    suffixes = r"(Inc|Ltd|Jr|Sr|Co|etc)"
    starters = r"(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
    acronyms = r"([A-Z][.][A-Z][.](?:[A-Z][.])?)"
    websites = r"[.](com|net|org|io|gov)"

    text = " " + texts + "  "
    text = text.replace("\n", " ")
    text = re.sub(prefixes, "\\1<prd>", text)
    text = re.sub(websites, "<prd>\\1", text)
    if "Ph.D" in text:
        text = text.replace("Ph.D.", "Ph<prd>D<prd>")
    text = re.sub(r"\s" + alphabets + "[.] ", " \\1<prd> ", text)
    text = re.sub(acronyms + " " + starters, "\\1<stop> \\2", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>\\3<prd>", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>", text)
    text = re.sub(" " + suffixes + "[.] " + starters, " \\1<stop> \\2", text)
    text = re.sub(" " + suffixes + "[.]", " \\1<prd>", text)
    text = re.sub(" " + alphabets + "[.]", " \\1<prd>", text)
    if "”" in text:
        text = text.replace(".”", "”.")
    if "\"" in text:
        text = text.replace(".\"", "\".")
    if "!" in text:
        text = text.replace("!\"", "\"!")
    if "?" in text:
        text = text.replace("?\"", "\"?")
    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

class Vectorizer:
    """
    pretrained_weights: str, default='distilbert-base-uncased'

        If the string does not include an extension .txt, .gz or .bin, then Bert vectorizer is loaded using the specified weights.
        Example: pass 'distilbert-base-multilingual-cased' to load Bert base multilingual model.

        To load word2vec vectorizer pass a valid path to the weights file (.txt, .gz or .bin).
        Example: pass 'glove-wiki-gigaword-300.gz' to load the Wiki vectors (when saved in the same folder you are running the code).

    
    ensemble_method: str, default='average'

        How word vectors are computed into sentece vectors.
    
    """
    def __init__(self, pretrained_weights = 'distilbert-base-uncased',
                       ensemble_method = 'average'):
        _, ext = os.path.splitext(pretrained_weights)
        self.vectors = []
        if not ext:
            print(f'Initializing Bert {pretrained_weights}')
            self.vectorizer = BertVectorizer(pretrained_weights=pretrained_weights)
        else:
            print(f'Initializing word2vec with vector path {pretrained_weights}')
            self.vectorizer = GensimVectorizer(pretrained_weights=pretrained_weights, 
                                               ensemble_method=ensemble_method)

    def run(self, sentences, remove_stop_words = ['not'], add_stop_words = []):
        # SANITY CHECK
        assert type(sentences) == list, 'A list must be passed!'
        for sentence in sentences:
            if type(sentence) != str:
                raise TypeError(f'All items must be string type but {sentence} is type {type(sentence)}.')
        # RUN
        self.vectorizer._execute(sentences, remove_stop_words=remove_stop_words, add_stop_words=add_stop_words)
        vectors = self.vectorizer.vectors
        for idx in range(vectors.shape[0]):
            self.vectors.append(vectors[idx])


class BaseVectorizer():
    def __init__(self, **kwargs):
        self.pretrained_weights = kwargs.get('pretrained_weights')
        self.ensemble_method = kwargs.get('ensemble_method')
        self.vectors = []
    
    def _load_model(self):
        pass

    def _execute(self):
        pass


class BertVectorizer(BaseVectorizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._load_model()
    
    def _load_model(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f'Vectorization done on {self.device}')
        model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel,
                                                            ppb.DistilBertTokenizer,
                                                            self.pretrained_weights)
        self.tokenizer = tokenizer_class.from_pretrained('../input/huggingface-bert/bert-base-uncased',local_files_only = True)
        self.model = model_class.from_pretrained('../input/huggingface-bert/bert-base-uncased',local_files_only = True)
    
    def _execute(self, sentences, **kwargs):
        model = self.model.to(self.device)
        model.eval()
        tokenized = list(map(lambda x: self.tokenizer.encode(x, add_special_tokens=True), sentences))
        max_len = 0
        for i in tokenized:
            if len(i) > max_len:
                max_len = len(i)
        padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])
        # Move inputs to same device as model
        input_ids = torch.tensor(np.array(padded)).type(torch.LongTensor).to(self.device)
        with torch.no_grad():
            last_hidden_states = model(input_ids)
        # Move vector results back to cpu if calculation was done on GPU
        vectors = last_hidden_states[0][:, 0, :].cpu().numpy()
        self.vectors = vectors

class GensimVectorizer(BaseVectorizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._load_model()
    
    def _load_model(self):
        _, file_extension = os.path.splitext(self.pretrained_weights)
        # Checks if file extension is binary
        if file_extension == '.bin':
            self.model = gensim.models.KeyedVectors.load_word2vec_format(self.pretrained_weights, binary=True)
        elif file_extension == '.txt' or '.gz':
            self.model = gensim.models.KeyedVectors.load_word2vec_format(self.pretrained_weights)
        else:
            raise IOError(f'The file extension {file_extension} is not valid. Word2vec valid formats are ".txt" and ".bin".')
    
    def _execute(self, sentences, **kwargs):
        splitter = Splitter()
        splitter.sent2words(sentences, remove_stop_words=kwargs.get('remove_stop_words'), add_stop_words=kwargs.get('add_stop_words'))
        words = splitter.words
        vectors = []
        for element in words:
            temp = []
            for w in element:
                temp.append(self.model[w])
            if self.ensemble_method == 'average':
                element_vec = np.mean(temp, axis=0)
                try:
                    vectors = np.vstack([vectors, element_vec])
                except:
                    vectors = element_vec
        self.vectors = vectors
        

In [62]:
# from sent2vec.vectorizer import Vectorizer
import numpy as np

def getVectors(discourses, dis_types):
    vectorizer = Vectorizer()
    dis_vectors = []
    for i in range(0,len(discourses),100):
        vectorizer.run(discourses[i:min(i+100,len(discourses))], remove_stop_words=[], add_stop_words=[])
        temp = vectorizer.vectors
        for t in range(i,min(i+100,len(discourses))):
            dis_vectors.append(temp[t][:100])
    temp = dis_vectors
    dis_vectors=[]
    for i in range(len(temp)):
        if (i==0):
            dis_vectors.append([*[0 for j in range(100)],*temp[i],*temp[i+1]])
        elif (i==len(temp)-1):
            dis_vectors.append([*temp[i-1],*temp[i],*[0 for j in range(100)]])
        else:
            dis_vectors.append([*temp[i-1],*temp[i],*temp[i+1]])
    dis_vectors = np.array(dis_vectors)      
    # concatenate the type[dis_types[i]] and dis_vectors
#     dis_vectors = np.concatenate((np.array([typeDict[dis_types[i]] for i in range(len(dis_types))]).reshape(-1,1),dis_vectors),axis=1)
        

#     print(len(dis_vectors))
#     print(dis_vectors[0])
    # return dis_vectors

    output1 = []
    output2 = []
    for i in range(0,len(dis_types)):
#         temp = [0 for i in range(0,len(typeDict))]
#         temp[typeDict[dis_types[i]]]=1
#         output1.append(temp)
        output1.append(typeDict[dis_types[i]])
    output1 = np.array(output1)
#     print(len(output1))
#     print(output1[0])
    # make the output as  the one hot encoding of the effectDict[dis_effectiveness[i]]
    for i in range(len(dis_effectiveness)):
#         temp = [0,0,0]
#         temp[effectDict[dis_effectiveness[i]]] = 1
#         output2.append(temp)
        output2.append(effectDict[dis_effectiveness[i]])
    output2 = np.array(output2)
#     print(len(output2))
#     print(output2[0])
    return dis_vectors,output1,output2

# train_input, train_output = getVectors(discourses, dis_types)
# getVectors(discourses, dis_types)

In [63]:
train_input, train_output_1, train_output_2 = getVectors(discourses, dis_types)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


Initializing Bert distilbert-base-uncased
Vectorization done on cuda


Some weights of the model checkpoint at ../input/huggingface-bert/bert-base-uncased were not used when initializing DistilBertModel: ['bert.encoder.layer.7.attention.self.query.bias', 'bert.encoder.layer.7.output.dense.bias', 'bert.encoder.layer.2.attention.self.query.bias', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.encoder.layer.4.attention.output.dense.bias', 'bert.encoder.layer.4.attention.self.key.bias', 'bert.encoder.layer.5.output.dense.bias', 'bert.encoder.layer.7.attention.output.LayerNorm.weight', 'bert.encoder.layer.4.output.dense.weight', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.3.attention.output.dense.weight', 'bert.encoder.layer.6.attention.self.key.bias', 'bert.encoder.layer.1.a

In [64]:
train_input.shape
# dum = [*[0 for j in range(100)],*train_input[0],*train_input[1]]
# len(dum)

(37762, 300)

In [65]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.

  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}

  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).

  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)
    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))

In [66]:
class_weights1 = generate_class_weights(train_output_1)
class_weights2 = generate_class_weights(train_output_2)
class_weights2

{0: 1.8367624884478817, 1: 0.5972354020370721, 2: 1.2801111902098377}

In [76]:
def weighted_categorical_crossentropy(class_weight):
    def loss(y_obs, y_pred):
        y_obs = tf.dtypes.cast(y_obs, tf.int32)
        hothot = tf.one_hot(tf.reshape(y_obs,[-1]),depth=len(class_weight))
#         weight = tf.math.multiply(class_weight, hothot)
#         weight = tf.reduce_sum(weight, axis=-1)
        losses = tf.compat.v1.losses.sparse_softmax_cross_entropy(
            labels=y_obs, logits=y_pred
        )
        return losses
    return loss

In [77]:
loss1 = weighted_categorical_crossentropy(list(class_weights1.values()))
loss2 = weighted_categorical_crossentropy(list(class_weights2.values()))

In [69]:
train_output_1.shape

(37762,)

In [87]:
# design a tensorflow keras classfication model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from math import ceil

# train_input = np.array(train_input)
# train_output = np.array(train_output)

# print(train_input.shape)
# print(train_output.shape)

# model = keras.Sequential([
#     layers.Dense(128, activation='relu', input_shape=[101]),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(3, activation='softmax')
# ])



def build_model():
    input_layer = layers.Input(shape=(300,))
    X = layers.Dense(128, activation='relu')(input_layer)
#     X = layers.Dropout(0.5)(X)
    X = layers.Dense(64, activation='relu')(X)
#     X = layers.Dropout(0.5)(X)
    output_layer_1 = layers.Dense(7,name="output_1")(X)
    output_layer_2 = layers.Dense(3,name="output_2")(output_layer_1)
    
    model = tf.keras.Model(inputs=[input_layer], outputs=[output_layer_1,output_layer_2])
    return model
model = build_model()
    

# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy'])

model.compile(
    optimizer='adam',
    loss={"output_1":loss1, "output_2": loss2},
    metrics={"output_1":loss1,"output_2":loss2}
)
train_output = [train_output_1, train_output_2]
# X_train, X_test, y_train, y_test = train_test_split(train_input, train_output, test_size=0.2)
X_train = train_input[:ceil(.8*len(train_input))]
y_train = [train_output_1[:ceil(.8*len(train_output_1))],train_output_2[:ceil(.8*len(train_output_2))]]
X_test = train_input[ceil(.8*len(train_input)):]
y_test = [train_output_1[ceil(.8*len(train_output_1)):],train_output_2[ceil(.8*len(train_output_2)):]]
model.fit(X_train, y_train, epochs=20, batch_size=15)
model.evaluate(X_test,y_test)

Epoch 1/20
2014/2014 [==============================] - 6s 3ms/step - loss: 2.4169 - output_1_loss: 1.4507 - output_2_loss: 0.9663
Epoch 2/20
2014/2014 [==============================] - 6s 3ms/step - loss: 2.2241 - output_1_loss: 1.2818 - output_2_loss: 0.9423
Epoch 3/20
2014/2014 [==============================] - 6s 3ms/step - loss: 2.1620 - output_1_loss: 1.2444 - output_2_loss: 0.9176
Epoch 4/20
2014/2014 [==============================] - 6s 3ms/step - loss: 2.1322 - output_1_loss: 1.2266 - output_2_loss: 0.9056
Epoch 5/20
2014/2014 [==============================] - 5s 3ms/step - loss: 2.1096 - output_1_loss: 1.2114 - output_2_loss: 0.8982
Epoch 6/20
2014/2014 [==============================] - 6s 3ms/step - loss: 2.0933 - output_1_loss: 1.2028 - output_2_loss: 0.8905
Epoch 7/20
2014/2014 [==============================] - 5s 3ms/step - loss: 2.0784 - output_1_loss: 1.1918 - output_2_loss: 0.8866
Epoch 8/20
2014/2014 [==============================] - 6s 3ms/step - loss: 2.0638 

NameError: name 'mode' is not defined

In [88]:
model.evaluate(X_test,y_test)

236/236 [==============================] - 1s 2ms/step - loss: 2.0933 - output_1_loss: 1.1028 - output_2_loss: 0.9906


[2.0933430194854736,
 1.1027781963348389,
 0.9905652403831482,
 1.1027781963348389,
 0.9905652403831482]

In [79]:
# create a submissions.csv file
# '''
# discourse_id,Ineffective,Adequate,Effective
# a261b6e14276,0.2,0.6,0.4
# 5a88900e7dc1,3.0,6.0,1.0
# 9790d835736b,1.0,2.0,3.0
# 75ce6d68b67b,0.33,0.34,0.33
# 93578d946723,0.01,0.24,0.47
# 2e214524dbe3,0.2,0.6,0.4
# 84812fc2ab9f,3.0,6.0,1.0
# c668ff840720,1.0,2.0,3.0
# 739a6d00f44a,0.33,0.34,0.33
# bcfae2c9a244,0.01,0.24,0.47
# '''

test_data = pd.read_csv('../input/feedback-prize-effectiveness/test.csv')
test_data.head()

test_sentences = test_data['discourse_text'].tolist()
# test_sentences[0]

test_dis_types  = test_data['discourse_type'].tolist()

test_input,_,_ = getVectors(test_sentences,test_dis_types)
test_input = np.array(test_input)

test_output = model.predict(test_input)
# test_output = np.argmax(test_output, axis=1)
# print(test_output)

import csv

with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["discourse_id","Ineffective","Adequate","Effective"])
    for i in range(len(test_output[1])):
#         temp = [0,0,0]
#         temp[test_output[i]] = 1
        writer.writerow([test_data['discourse_id'][i],test_output[1][i][0],test_output[1][i][1],test_output[1][i][2]])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


Initializing Bert distilbert-base-uncased
Vectorization done on cuda


Some weights of the model checkpoint at ../input/huggingface-bert/bert-base-uncased were not used when initializing DistilBertModel: ['bert.encoder.layer.7.attention.self.query.bias', 'bert.encoder.layer.7.output.dense.bias', 'bert.encoder.layer.2.attention.self.query.bias', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.encoder.layer.4.attention.output.dense.bias', 'bert.encoder.layer.4.attention.self.key.bias', 'bert.encoder.layer.5.output.dense.bias', 'bert.encoder.layer.7.attention.output.LayerNorm.weight', 'bert.encoder.layer.4.output.dense.weight', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.3.attention.output.dense.weight', 'bert.encoder.layer.6.attention.self.key.bias', 'bert.encoder.layer.1.a

In [ ]:
# test_output[1][0][0]

In [ ]:
# get accuracy of the model
# from sklearn.metrics import accuracy_score
# print(accuracy_score(train_output, model.predict(train_input)))

In [ ]:
# import spacy

In [ ]:
#define losses according to respective class weights
#build model with two output layers
#redifine get vectors, it will now return two output matrices
#compil model
#train